In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as colors
pio.templates.default = "plotly_white"

In [2]:
df = pd.read_csv("Sample - Superstore.csv", encoding = 'latin-1')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
df.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [4]:
df.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [5]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

In [6]:
df['Order Month'] = df['Order Date'].dt.month
df['Order Year'] = df['Order Date'].dt.year
df['Order DOW'] = df['Order Date'].dt.dayofweek

In [12]:
#Monthly Sales Analysis
month_sales = df.groupby('Order Month')['Sales'].sum().reset_index()
fig = px.line(month_sales,
             x='Order Month',
             y='Sales',
             title = 'Monthly Sales')
fig.show()

In [14]:
#Sales by category
category_sales = df.groupby('Category')['Sales'].sum().reset_index()
fig = px.pie(category_sales,
            values='Sales',
            names='Category',
            hole=0.5,
            color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title_text = 'Sales Category', title_font=dict(size=24))
fig.show()

In [19]:
#Sales by sub-category
sub_category_sales = df.groupby('Sub-Category')['Sales'].sum().reset_index()
fig = px.bar(sub_category_sales, x='Sub-Category', y='Sales', title = "Sub_category Sales")
fig.show()

In [21]:
#Monthly Profit
month_profit = df.groupby('Order Month')['Profit'].sum().reset_index()
fig = px.line(month_profit, x='Order Month', y='Profit', title='Monthly Profit')
fig.show()

In [22]:
#Profit by sub-category
sub_category_profit = df.groupby('Sub-Category')['Profit'].sum().reset_index()
fig = px.bar(sub_category_profit, x= 'Sub-Category', y = 'Profit', title = 'Sub-Category Profit')
fig.show()

In [23]:
#Sales and profit by customer segment
s_p_customer = df.groupby('Segment').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()
color_pallete = colors.qualitative.Pastel
fig = go.Figure()
fig.add_trace(go.Bar(x=s_p_customer['Segment'],
                    y=s_p_customer['Sales'],
                    name='Sales',
                    marker_color = color_pallete[0]))
fig.add_trace(go.Bar(x=s_p_customer['Segment'],
                    y=s_p_customer['Profit'],
                    name='Profit',
                    marker_color = color_pallete[1]))
fig.update_layout(title='Sales and Profit by segment', xaxis_title = 'Customer Segment', yaxis_title = 'Amount')
fig.show()

In [25]:
#Sales to profit ratio
ratio = df.groupby('Segment').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()
ratio['Sales_profit_ratio'] = ratio['Sales'] / ratio['Profit']
print(ratio[['Segment', 'Sales_profit_ratio']])

       Segment  Sales_profit_ratio
0     Consumer            8.659471
1    Corporate            7.677245
2  Home Office            7.125416
